In [1]:
import tensorflow as tf
from tqdm import tqdm

from deep_lss.utils import distribute
from msfm.fiducial_pipeline import FiducialPipeline

23-08-24 03:16:49    scales.py INF   Setting up healpy to run on 128 CPUs 


In [2]:
tfr_pattern = "/pscratch/sd/a/athomsen/DESY3/v3/fiducial/DESy3_fiducial_???.tfrecord"
local_batch_size = 16
n_steps = 300
multi_noise = True

In [3]:
fiducial_pipeline = FiducialPipeline(
    # params=["Om", "s8"],
    # params=["Om", "s8", "bg", "n_bg"],
    # params=["Om", "s8", "Aia", "n_Aia"],
    params=["Om", "s8", "Aia", "n_Aia", "bg", "n_bg"],
    with_lensing=True,
    with_clustering=True,
    apply_norm=True,
)

dset_kwargs = {
    "tfr_pattern": tfr_pattern,
    "local_batch_size": local_batch_size,
    "is_cached": False,
    "n_readers": 16,
    # "n_readers": 1,
    "n_prefetch": 0,
    # "n_prefetch": 10,
    # "is_eval": False,
    "file_name_shuffle_buffer": 16,
    "examples_shuffle_buffer": None,
}



23-08-24 03:16:50     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_512.h5 
23-08-24 03:16:50     files.py INF   Loaded the pixel file /global/u2/a/athomsen/multiprobe-simulation-forward-model/data/DESY3_pixels_512.h5 


# local

In [4]:
if multi_noise:
    print("multi noise")
    dset = fiducial_pipeline.get_multi_noise_dset(
        n_noise=3,
        **dset_kwargs,
    )

else:
    print("single noise")
    dset = fiducial_pipeline.get_dset(
        i_noise=0,
        **dset_kwargs,
    )
    
dset_iter = iter(dset)

multi noise
23-08-24 03:16:53 fiducial_pip INF   Shuffling file names 
23-08-24 03:16:54 tfrecords.py WAR   Tracing parse_inverse_fiducial 
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
23-08-24 03:16:55 fiducial_pip INF   Batching into 16 elements locally 
23-08-24 03:16:55 fiducial_pip WAR   Tracing _

# with strategy

In [5]:
# def dataset_fn(input_context):
#     if multi_noise:
#         print("multi noise")
#         dset = fiducial_pipeline.get_multi_noise_dset(
#             n_noise=3,
#             **dset_kwargs,
#             input_context=input_context,
#         )

#     else:
#         print("single noise")
#         dset = fiducial_pipeline.get_dset(
#             i_noise=0,
#             **dset_kwargs,
#             input_context=input_context,
#         )

#     return dset

# strategy = distribute.get_strategy(False)
# dset = strategy.distribute_datasets_from_function(dataset_fn)
# dset_iter = iter(dset)

# for both

In [ ]:
for step in tqdm(range(n_steps), total=n_steps):
    _, _ = next(dset_iter)
    
print("terminated successfully")

  0%|          | 1/300 [00:30<2:31:36, 30.42s/it]